In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ycb import ycb_colourspace_stuff as col
# TODO: See gpt about detecting skin tone in ycb


Load image

In [ ]:
image_path = "./outputs/example_seg_img.png"
img_bgr = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
image_ycb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2YCrCb)
y, cr, cb = cv2.split(image_ycb)
cb = cb.astype(np.float32)
cr = cr.astype(np.float32)


Detect greenish pixels

In [ ]:
spill_mask = ((cb > 120) & (cr < 150)).astype(np.float32)
spill_mask = cv2.GaussianBlur(spill_mask, (7, 7), 0)


In [ ]:
print(f"Green spill coverage: {np.mean(spill_mask) * 100:.2f}%")

Apply correction globally

In [ ]:
correction_strength = 2.0
cb -= correction_strength * spill_mask * (cb - 128)
cr += correction_strength * spill_mask * (128 - cr)


Clip and convert back

In [ ]:
cb = np.clip(cb, 0, 255).astype(np.uint8)
cr = np.clip(cr, 0, 255).astype(np.uint8)

corrected_ycb = cv2.merge((y, cr, cb))
corrected_bgr = cv2.cvtColor(corrected_ycb, cv2.COLOR_YCrCb2BGR)
corrected_rgb = cv2.cvtColor(corrected_bgr, cv2.COLOR_BGR2RGB)


Display before and after green spill correction

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(corrected_rgb)
plt.title("Corrected Image")
plt.axis('off')

plt.tight_layout()
plt.show()

Calculate green

In [ ]:
total_og_green = col.quantify_green(img_ycb=image_ycb)
total_corrected_green = col.quantify_green(corrected_ycb)


In [ ]:
print(f"Total OG Green: {total_og_green}")  # 2,360
print(f"Total Corrected Green: {total_corrected_green}")    # 27
